In [ ]:
import numpy as np
import matplotlib.pylab as plt

## Hump Geometry

![Figs/Hump_formula_geometry.png](Figs/Hump_formula_geometry.png)

### Define hump function

In [ ]:
# Another way of defining the bump function
def define_bump_geometry():
    tau = 0.1
    yc = (tau**2 - 0.25) / (2 * tau)
    R2 = 0.25 + yc**2
    R = np.sqrt(R2)

    def bump(x):
        return yc + np.sqrt(R2 - (x - 0.5)**2)
    
    return bump

bump_func=define_bump_geometry()

### Generate the coordinates for all components

In [ ]:
# Generate bump coordinates
x_bump = np.linspace(0, 1, 500)
y_bump = bump_func(x_bump)

# Boundary points
left_x = [-1, -1]
left_y = [0, 1]

bottom_left_x = [-1, 0]
bottom_left_y = [0, 0]

bottom_right_x = [1, 2]
bottom_right_y = [0, 0]

right_x = [2, 2]
right_y = [0, 1]

top_x = [-1, 2]
top_y = [1, 1]

# Concatenate all coordinates
x_all = np.concatenate([left_x, bottom_left_x, x_bump, bottom_right_x, right_x, top_x])
y_all = np.concatenate([left_y, bottom_left_y, y_bump, bottom_right_y, right_y, top_y])

### Plotting

In [ ]:
# Plot
plt.figure(figsize=(10, 5))

# Plot concatenated line
plt.plot(x_all, y_all, color='black')

# Set axis limits
plt.xlim(-1.2, 2.25)
plt.ylim(-0.2, 1.25)
plt.gca().set_aspect('equal', adjustable='box')

# Set labels for boundaries
plt.text(-1.12, 0.5, 'Inlet Boundary', verticalalignment='center', color='blue', rotation=90)
plt.text(2.1, 0.5, 'Outlet Boundary', verticalalignment='center', color='red', rotation=90)
plt.text(0.5, 1.05, 'Wall Boundary', horizontalalignment='center', verticalalignment='bottom', color='green')

plt.savefig('hump_geometry.pdf')

___
## Generate The Grid

In [ ]:
#simplify the domain/number of grids specification

# Define the domain: [x0, xend, y0, yend, bump_x0, bump_xn]
domain = [-1, 2, 0.1, 1.5, 0, 1]

# Specify the number of grid points
nx1= 20;
nxb= 30;
nx2= 20;
ny=20;
npoints=[nx1, nxb, nx2, ny]

In [ ]:
# Generate points with bump boundary
def generate_points_with_bump_rect(domain, npoints, bump_func):
    x0, xend, y0, yend, bump_x0, bump_xn = domain
    nx1, nxb, nx2, ny = npoints

    #x coordinates
    x_linear_1 = np.linspace(x0, bump_x0, nx1)
    x_bump = np.linspace(bump_x0, bump_xn, nxb)
    x_linear_2 = np.linspace(bump_xn, xend, nx2)
    x1d=np.hstack([x_linear_1, x_bump, x_linear_2])
    nx=len(x1d)
    
    #define the bottom and top wall's grids
    y_bottom_1=np.zeros_like(x_linear_1)+y0
    y_bottom_b=bump_func(x_bump)+y0
    y_bottom_2=np.zeros_like(x_linear_2)+y0
    y_bottom=np.concatenate([y_bottom_1, y_bottom_b, y_bottom_2])
    y_top=np.zeros_like(y_bottom)+(yend)

    #eta: a uniform grid from 0 to 1, divided by the number of grid points in y
    eta=np.linspace(0, 1, ny)

    #2d variables
    X2D,Y2D=np.meshgrid(x1d,eta)
    X2D=X2D.T
    Y2D=Y2D.T

    local_height=y_top-y_bottom
    for i in range(nx):
        Y2D[i,:]=local_height[i]*eta+y_bottom[i]

    return X2D,Y2D


In [ ]:
bump_func=define_bump_geometry()
X2D,Y2D=generate_points_with_bump_rect(domain, npoints, bump_func)

fig, ax=plt.subplots(1,1,figsize=(10,4))
plt.plot(X2D,Y2D,color='black');
plt.plot(X2D.T,Y2D.T,color='red');

### Add a grid clustering in y-direction

In [ ]:
# Generate points with bump boundary
def generate_points_with_bump_rect_betay(domain, npoints, bump_func, betay=2.):
    x0, xend, y0, yend, bump_x0, bump_xn = domain
    nx1, nxb, nx2, ny = npoints

    #x coordinates
    x_linear_1 = np.linspace(x0, bump_x0, nx1)
    x_bump = np.linspace(bump_x0, bump_xn, nxb)
    x_linear_2 = np.linspace(bump_xn, xend, nx2)
    x1d=np.hstack([x_linear_1, x_bump, x_linear_2])
    nx=len(x1d)
    
    #define the bottom and top wall's grids
    y_bottom_1=np.zeros_like(x_linear_1)+y0
    y_bottom_b=bump_func(x_bump)+y0
    y_bottom_2=np.zeros_like(x_linear_2)+y0
    y_bottom=np.concatenate([y_bottom_1, y_bottom_b, y_bottom_2])
    y_top=np.zeros_like(y_bottom)+(yend)

    # eta: a uniform grid from -1 to 1, divided by the number of grid points in y
    eta=np.linspace(-1, 1, ny)
    # apply tanh clustering to eta
    eta=np.tanh(betay*eta)/np.tanh(betay)
    # convert eta to something between 0 and 1, like before
    eta=(eta+1.)/2.
    
    #2d variables
    X2D,Y2D=np.meshgrid(x1d,eta)
    X2D=X2D.T
    Y2D=Y2D.T

    local_height=y_top-y_bottom
    for i in range(nx):
        Y2D[i,:]=local_height[i]*eta+y_bottom[i]
    
    return X2D,Y2D


In [ ]:
bump_func=define_bump_geometry()
X2D,Y2D=generate_points_with_bump_rect_betay(domain, npoints, bump_func,betay=1.5)

fig, ax=plt.subplots(1,1,figsize=(10,4))
plt.plot(X2D,Y2D,color='black');
plt.plot(X2D.T,Y2D.T,color='red');

### Add grid clustering in x-direction near the bump

In [ ]:
def generate_points_with_bump_rect_betay(domain, npoints, bump_func, betax=2., betay=1.5):
    x0, xend, y0, yend, bump_x0, bump_xn = domain
    nx1, nxb, nx2, ny = npoints

    #x coordinates

    # x_linear_1
    xi_linear_1=np.linspace(0,1,nx1) #[0,1] uniform
    x_linear_1=(np.tanh(betax*xi_linear_1)/np.tanh(betax)) #[0,1] and clustered near 1
    x_linear_1=x_linear_1*(bump_x0-x0)+x0

    #x_bump
    xi_bump=np.linspace(-1,1,nxb) #[-1,1] and uniform
    x_bump=np.tanh(betax*xi_bump)/np.tanh(betax) #[-1,1] and clustered near both -1 and 1
    x_bump=((x_bump+1.)/2.) #[0,1] and clustered near both 0 and 1
    x_bump=x_bump*(bump_xn-bump_x0)+bump_x0
    x_bump=np.copy(x_bump[1:-1]) #there will be an overlap between the first point of x_bump and last point of x_linear_1
    
    #x_linear2
    xi_linear_2=np.linspace(-1, 0, nx2) #[-1,0] and uniform
    x_linear_2=(np.tanh(betax*xi_linear_2)/np.tanh(betax)) #[-1,0] and clustered near 1
    x_linear_2=x_linear_2+1. #[0,1] and clustered near 0
    x_linear_2=x_linear_2*(xend-bump_xn)+bump_xn
    
    #put it all together
    x1d=np.hstack([x_linear_1, x_bump, x_linear_2])
    nx=len(x1d)
    
    #define the bottom and top wall's grids
    y_bottom_1=np.zeros_like(x_linear_1)+y0
    y_bottom_b=bump_func(x_bump)+y0
    y_bottom_2=np.zeros_like(x_linear_2)+y0
    y_bottom=np.concatenate([y_bottom_1, y_bottom_b, y_bottom_2])
    y_top=np.zeros_like(y_bottom)+(yend)

    # eta: a uniform grid from -1 to 1, divided by the number of grid points in y
    eta=np.linspace(-1, 1, ny)
    # apply tanh clustering to eta
    eta=np.tanh(betay*eta)/np.tanh(betay)
    # convert eta to something between 0 and 1, like before
    eta=(eta+1.)/2.
    
    #2d variables
    X2D,Y2D=np.meshgrid(x1d,eta)
    X2D=X2D.T
    Y2D=Y2D.T

    local_height=y_top-y_bottom
    for i in range(nx):
        Y2D[i,:]=local_height[i]*eta+y_bottom[i]
    
    return X2D,Y2D


In [ ]:
bump_func=define_bump_geometry()
X2D,Y2D=generate_points_with_bump_rect_betay(domain, npoints, bump_func,betay=1.5,betax=1.1)

fig, ax=plt.subplots(1,1,figsize=(10,4))
plt.plot(X2D,Y2D,color='black');
plt.plot(X2D.T,Y2D.T,color='red');